In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date, time
from app import db
from db import DeviceInfo, DeadManPackage, Dashboard, InstructionPackage, Connectivity, PresenceDetectorStatistics
from db import VersionPackage, SensorData, TemperaturePackage, PressurePackage, HumidityPackage, BrightnessPackage, GasPackage
from utils.interval import intersect_intervals, remove_short_intervals, TimeInterval

from analytics.sensors import Sensor, _get_sensor_data, resample_sensor_data, get_sensor_data
from analytics.instruction import get_state_data, get_instructions_data
from utils.interval import find_intervals
from analytics.scenes import get_scene_durations, get_settings_durations

In [4]:
pd.set_option('display.max_rows', 110)

day1 = datetime.strptime('2020-03-12 00:00:00', '%Y-%m-%d %H:%M:%S')
day2 = datetime.strptime('2020-03-13 00:00:00', '%Y-%m-%d %H:%M:%S')

#since = datetime.now() - timedelta(days=4)
#until = datetime.now() - timedelta(days=2)

since = datetime.strptime('2020-03-01 00:00:00', '%Y-%m-%d %H:%M:%S')
until = datetime.strptime('2020-04-14 00:00:00', '%Y-%m-%d %H:%M:%S')
device = "PTL_RD_ES_003"

In [6]:
data = get_scene_durations(device, since.date(), until.date())

/usr/lib/python3/dist-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/lib/python3/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/lib/python3/dist-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pa

In [7]:
data

,AUTO,LIGHT_SHOWER,TASK_HORI,TASK_VERT
2020-03-11,02:36:02,00:00:02,00:00:01,00:38:22
2020-03-12,02:55:07,00:00:00,00:00:00,00:00:00
2020-03-13,00:24:15,00:00:00,00:00:00,00:00:00
2020-03-14,00:11:12,00:00:00,00:00:00,00:00:00
2020-03-15,00:21:56,00:00:00,00:00:00,00:00:00
2020-03-16,00:26:58,00:00:00,00:00:00,00:00:00
2020-03-17,00:13:45,00:00:00,00:00:00,00:00:00
2020-03-18,00:21:56,00:00:00,00:00:00,00:00:00
2020-03-19,00:20:20,00:00:00,00:00:00,00:00:00
2020-03-20,00:20:11,00:00:00,00:00:00,00:00:00


In [5]:
s_data = settings_durations(device, since.date(), until.date())

/usr/lib/python3/dist-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/lib/python3/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [7]:
pd.DataFrame(s_data)

,count,duration
begin,,
2020-03-11,2,00:02:02
2020-03-12,4,00:02:11


In [14]:
da = data['AUTO']
da.groupby([da.begin.dt.date]).duration.sum()

begin
2020-03-02   10:40:44
2020-03-03   15:41:57
2020-03-04   18:05:11
2020-03-05   08:52:40
2020-03-06   00:01:04
2020-03-07   00:02:39
2020-03-09   09:22:54
2020-03-10   07:57:28
2020-03-11   06:53:47
2020-03-12   09:03:53
2020-03-13   05:14:43
2020-03-14   00:01:45
2020-03-15   00:10:46
2020-03-16   00:14:14
2020-03-17   00:40:01
2020-03-18   00:26:40
2020-03-19   00:27:32
2020-03-20   00:24:05
2020-03-21   00:31:29
2020-03-22   00:24:41
2020-03-23   00:41:15
2020-03-24   00:33:54
2020-03-25   00:05:29
2020-03-26   00:17:34
2020-03-27   00:15:24
2020-03-28   00:13:30
2020-03-29   00:17:34
2020-03-30   00:15:48
2020-03-31   00:01:35
2020-04-01   00:02:20
2020-04-02   00:03:39
2020-04-03   00:00:27
2020-04-04   00:02:00
2020-04-05   00:00:41
2020-04-06   00:01:28
2020-04-07   00:03:56
2020-04-08   00:00:46
2020-04-09   00:01:03
2020-04-10   00:00:28
2020-04-11   00:00:29
2020-04-12   00:00:29
2020-04-13   00:00:59
2020-04-14   00:06:50
Name: duration, dtype: timedelta64[ns]

In [38]:
%%time
data_instructions = get_instructions_data(device, (since, until))
data = get_state_data(device, (since, until), resample_rule='1S')
data

CPU times: user 5.86 s, sys: 681 ms, total: 6.54 s
Wall time: 7.01 s


,power,scene,settings
timestamp,,,
2020-01-14 09:53:01,OFF,AUTO,OFF
2020-01-14 09:53:02,OFF,AUTO,OFF
2020-01-14 09:53:03,OFF,AUTO,OFF
2020-01-14 09:53:04,OFF,AUTO,OFF
2020-01-14 09:53:05,OFF,AUTO,OFF
2020-01-14 09:53:06,OFF,AUTO,OFF
2020-01-14 09:53:07,OFF,AUTO,OFF
2020-01-14 09:53:08,OFF,AUTO,OFF
2020-01-14 09:53:09,OFF,AUTO,OFF


In [9]:
find_intervals(data[(data.power == 'ON') & (data.scene == "LIGHT_SHOWER") & (data.settings=="OFF")].reset_index())

/usr/lib/python3/dist-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/lib/python3/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,begin,power,scene,settings,duration,end
0,2020-01-15 11:25:56,ON,LIGHT_SHOWER,OFF,00:00:01,2020-01-15 11:25:57
1,2020-01-15 11:25:59,ON,LIGHT_SHOWER,OFF,00:00:01,2020-01-15 11:26:00
2,2020-01-15 11:26:49,ON,LIGHT_SHOWER,OFF,00:00:01,2020-01-15 11:26:50
3,2020-01-15 11:26:53,ON,LIGHT_SHOWER,OFF,00:00:01,2020-01-15 11:26:54
4,2020-01-15 13:40:24,ON,LIGHT_SHOWER,OFF,00:00:01,2020-01-15 13:40:25
5,2020-01-17 11:37:03,ON,LIGHT_SHOWER,OFF,00:00:01,2020-01-17 11:37:04
6,2020-01-17 11:42:39,ON,LIGHT_SHOWER,OFF,00:00:01,2020-01-17 11:42:40
7,2020-01-21 15:37:08,ON,LIGHT_SHOWER,OFF,00:00:05,2020-01-21 15:37:13
8,2020-01-21 15:37:17,ON,LIGHT_SHOWER,OFF,00:00:01,2020-01-21 15:37:18
9,2020-01-21 15:37:42,ON,LIGHT_SHOWER,OFF,00:00:01,2020-01-21 15:37:43


In [4]:
data_settings = data[(data.power == 'ON') & (data.settings == 'ON')] \
    .reset_index()
len(data_settings)

21785

In [18]:
d1 = day1.date()

TypeError: unsupported operand type(s) for -: 'datetime.time' and 'datetime.timedelta'

In [37]:
from datetime import datetime
datetime.combine(d1, time(23, 59, 59, 999999))

datetime.datetime(2020, 3, 12, 23, 59, 59, 999999)

In [5]:
find_intervals(data_settings)

/usr/lib/python3/dist-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/lib/python3/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,begin,power,scene,settings,duration,end
0,2020-01-15 11:26:01,ON,LIGHT_SHOWER,ON,00:00:47,2020-01-15 11:26:48
1,2020-01-15 11:27:17,ON,TASK_VERT,ON,00:00:43,2020-01-15 11:28:00
2,2020-01-15 12:45:33,ON,AUTO,ON,00:02:26,2020-01-15 12:47:59
3,2020-01-21 15:39:28,ON,AUTO,ON,00:00:15,2020-01-21 15:39:43
4,2020-01-21 15:39:48,ON,AUTO,ON,00:00:10,2020-01-21 15:39:58
5,2020-01-21 15:40:02,ON,AUTO,ON,00:01:06,2020-01-21 15:41:08
6,2020-01-21 15:41:19,ON,AUTO,ON,00:00:24,2020-01-21 15:41:43
7,2020-01-21 15:42:00,ON,AUTO,ON,00:00:53,2020-01-21 15:42:53
8,2020-01-21 15:42:55,ON,AUTO,ON,00:00:49,2020-01-21 15:43:44
9,2020-01-21 15:43:49,ON,AUTO,ON,00:00:24,2020-01-21 15:44:13


In [194]:
ds = data_settings
ds['delay'] = ds.timestamp.diff()
ds['new_interval'] = (ds.delay > timedelta(seconds=1)).astype(int)
ds.loc[0, 'new_interval'] = 1
ds['keep_row'] = ds.new_interval.diff(periods=-1)
ds['single_element_interval'] = (ds.new_interval.rolling(2).sum() >= 2.0).astype(int).shift(-1)
ds.loc[ds.keep_row.isna(), 'keep_row'] = -1.0
ds = ds[(ds.keep_row != 0.0) | (ds.single_element_interval != 0.0)]
ds = ds.rename(columns=dict(timestamp='begin'))
ds['duration'] = ds.begin.diff(periods=-1).abs()
ds.loc[ds.single_element_interval != 0, 'duration'] = timedelta(seconds=1)
ds = ds[ds.new_interval == 1]
ds['end'] = ds.begin + ds.duration
ds = ds.reset_index(drop=True)
ds = ds.drop(columns=['delay', 'new_interval', 'keep_row', 'single_element_interval'])

In [195]:
#ds[ds.keep_row != 0.0]
ds

,begin,power,scene,settings,duration,end
0,2020-01-15 11:26:01,ON,LIGHT_SHOWER,ON,00:00:47,2020-01-15 11:26:48
1,2020-01-15 11:27:17,ON,TASK_VERT,ON,00:00:43,2020-01-15 11:28:00
2,2020-01-15 12:45:33,ON,AUTO,ON,00:02:26,2020-01-15 12:47:59
3,2020-01-21 15:39:28,ON,AUTO,ON,00:00:15,2020-01-21 15:39:43
4,2020-01-21 15:39:48,ON,AUTO,ON,00:00:10,2020-01-21 15:39:58
5,2020-01-21 15:40:02,ON,AUTO,ON,00:01:06,2020-01-21 15:41:08
6,2020-01-21 15:41:19,ON,AUTO,ON,00:00:24,2020-01-21 15:41:43
7,2020-01-21 15:42:00,ON,AUTO,ON,00:00:53,2020-01-21 15:42:53
8,2020-01-21 15:42:55,ON,AUTO,ON,00:00:49,2020-01-21 15:43:44
9,2020-01-21 15:43:49,ON,AUTO,ON,00:00:24,2020-01-21 15:44:13


In [196]:
day1 = datetime.strptime('2020-01-21 15:54:00', '%Y-%m-%d %H:%M:%S')
day2 = datetime.strptime('2020-01-22 10:00:00', '%Y-%m-%d %H:%M:%S')

ds[(ds.begin > day1) & (ds.begin < day2)]

,begin,power,scene,settings,duration,end
20,2020-01-21 15:54:23,ON,LIGHT_SHOWER,ON,00:00:07,2020-01-21 15:54:30
21,2020-01-21 15:55:08,ON,TASK_VERT,ON,00:00:01,2020-01-21 15:55:09
22,2020-01-22 09:28:12,ON,AUTO,ON,00:01:18,2020-01-22 09:29:30
23,2020-01-22 09:29:36,ON,AUTO,ON,00:00:14,2020-01-22 09:29:50
24,2020-01-22 09:29:53,ON,AUTO,ON,00:00:05,2020-01-22 09:29:58
25,2020-01-22 09:30:13,ON,TASK_VERT,ON,00:00:03,2020-01-22 09:30:16
26,2020-01-22 09:30:39,ON,AUTO,ON,00:00:25,2020-01-22 09:31:04
27,2020-01-22 09:31:08,ON,AUTO,ON,00:00:02,2020-01-22 09:31:10
28,2020-01-22 09:31:12,ON,AUTO,ON,00:00:06,2020-01-22 09:31:18
29,2020-01-22 09:31:21,ON,AUTO,ON,00:00:01,2020-01-22 09:31:22


In [192]:
ds[ds.single_element_interval != 0]

,begin,power,scene,settings,delay,new_interval,keep_row,single_element_interval,duration
756,2020-01-21 15:55:08,ON,TASK_VERT,ON,00:00:38,1,0.0,1.0,00:00:01
897,2020-01-22 09:31:21,ON,AUTO,ON,00:00:03,1,0.0,1.0,00:00:01
1040,2020-01-22 09:49:37,ON,AUTO,ON,00:01:07,1,0.0,1.0,00:00:01


In [66]:
ds_flat = ds

In [ ]:
ds['single_element_interval'] = (ds.new_interval.rolling(2).sum() >= 2.0).astype(int).shift(-1)
ds.loc[ds.keep_row.isna(), 'keep_row'] = -1.0
ds = ds[(ds.keep_row != 0.0) | (ds.single_element_interval != 0.0)]

In [209]:
ds = data_settings
ds['delay'] = ds.groupby([ds.timestamp.dt.date]).timestamp.diff()
ds['new_interval'] = (ds.delay > timedelta(seconds=1)).astype(int)
ds.loc[ds.delay.isna(), 'new_interval'] = 1
ds['keep_row'] = ds.groupby([ds.timestamp.dt.date]).new_interval.diff(periods=-1)
#ds['single_element_interval'] = (ds.new_interval.rolling(2).sum() >= 2.0).astype(int).shift(-1)
#ds.loc[ds.keep_row.isna(), 'keep_row'] = -1.0
#ds = ds[(ds.keep_row != 0.0) | (ds.single_element_interval != 0.0)]
#ds = ds.rename(columns=dict(timestamp='begin'))
#ds['duration'] = ds.groupby([ds.begin.dt.date]).begin.diff(periods=-1).abs()
#ds.loc[ds.single_element_interval != 0, 'duration'] = timedelta(seconds=1)
#ds = ds[ds.new_interval == 1]
#ds['end'] = ds.begin + ds.duration
#ds = ds.reset_index(drop=True)
#ds = ds.drop(columns=['delay', 'new_interval', 'keep_row', single_element_interval])
ds

,timestamp,power,scene,settings,delay,new_interval,keep_row
0,2020-01-15 11:26:01,ON,LIGHT_SHOWER,ON,NaT,1,1.0
1,2020-01-15 11:26:02,ON,LIGHT_SHOWER,ON,00:00:01,0,0.0
2,2020-01-15 11:26:03,ON,LIGHT_SHOWER,ON,00:00:01,0,0.0
3,2020-01-15 11:26:04,ON,LIGHT_SHOWER,ON,00:00:01,0,0.0
4,2020-01-15 11:26:05,ON,LIGHT_SHOWER,ON,00:00:01,0,0.0
5,2020-01-15 11:26:06,ON,LIGHT_SHOWER,ON,00:00:01,0,0.0
6,2020-01-15 11:26:07,ON,LIGHT_SHOWER,ON,00:00:01,0,0.0
7,2020-01-15 11:26:08,ON,LIGHT_SHOWER,ON,00:00:01,0,0.0
8,2020-01-15 11:26:09,ON,LIGHT_SHOWER,ON,00:00:01,0,0.0
9,2020-01-15 11:26:10,ON,LIGHT_SHOWER,ON,00:00:01,0,0.0


In [208]:
day1 = datetime.strptime('2020-01-22 00:00:00', '%Y-%m-%d %H:%M:%S')
day2 = datetime.strptime('2020-01-23 00:00:00', '%Y-%m-%d %H:%M:%S')

ds[(ds.timestamp > day1) & (ds.timestamp < day2)]

,timestamp,power,scene,settings,delay
757,2020-01-22 09:28:12,ON,AUTO,ON,NaT
758,2020-01-22 09:28:13,ON,AUTO,ON,00:00:01
759,2020-01-22 09:28:14,ON,AUTO,ON,00:00:01
760,2020-01-22 09:28:15,ON,AUTO,ON,00:00:01
761,2020-01-22 09:28:16,ON,AUTO,ON,00:00:01
762,2020-01-22 09:28:17,ON,AUTO,ON,00:00:01
763,2020-01-22 09:28:18,ON,AUTO,ON,00:00:01
764,2020-01-22 09:28:19,ON,AUTO,ON,00:00:01
765,2020-01-22 09:28:20,ON,AUTO,ON,00:00:01
766,2020-01-22 09:28:21,ON,AUTO,ON,00:00:01


In [107]:
ds[ds.keep_row.isna()]

,timestamp,power,scene,settings,delay,new_interval,keep_row
238,2020-01-15 12:47:59,ON,AUTO,ON,00:00:01,0,NaN
756,2020-01-21 15:55:08,ON,TASK_VERT,ON,00:00:38,1,NaN
1347,2020-01-22 13:40:48,ON,TASK_HORI,ON,00:00:01,0,NaN
11762,2020-01-27 18:14:17,ON,TASK_HORI,ON,00:00:01,0,NaN
18723,2020-01-28 15:19:41,ON,AUTO,ON,00:00:01,0,NaN
18901,2020-01-30 10:22:58,ON,AUTO,ON,00:00:01,0,NaN
19002,2020-01-31 12:37:51,ON,AUTO,ON,00:00:01,0,NaN
19689,2020-02-03 12:40:35,ON,AUTO,ON,00:00:01,0,NaN
21413,2020-02-05 17:58:20,ON,AUTO,ON,00:00:01,0,NaN
21434,2020-02-06 15:51:12,ON,AUTO,ON,00:00:01,0,NaN


In [92]:
ds[ds.delay.isna()]

,timestamp,power,scene,settings,delay
0,2020-01-15 11:26:01,ON,LIGHT_SHOWER,ON,NaT
239,2020-01-21 15:39:28,ON,AUTO,ON,NaT
757,2020-01-22 09:28:12,ON,AUTO,ON,NaT
1348,2020-01-27 11:21:38,ON,AUTO,ON,NaT
11763,2020-01-28 09:14:35,ON,TASK_HORI,ON,NaT
18724,2020-01-30 10:17:20,ON,LIGHT_SHOWER,ON,NaT
18902,2020-01-31 12:36:11,ON,AUTO,ON,NaT
19003,2020-02-03 11:23:21,ON,AUTO,ON,NaT
19690,2020-02-05 10:55:14,ON,TASK_HORI,ON,NaT
21414,2020-02-06 15:50:52,ON,AUTO,ON,NaT


In [73]:
flat = list(ds_flat.begin)
grouped = list(ds_grouped.begin)

In [76]:
for a,b in zip(flat, grouped):
    if a==b:
        continue
    else:
        print(a)
        print(b)
        break

2020-01-22 09:28:12
2020-01-21 15:55:08


In [27]:
%%time
ddd = data.reset_index()
ddd['date'] = ddd.timestamp.apply(lambda t: t.date())
ddd

CPU times: user 13.4 s, sys: 7.31 s, total: 20.7 s
Wall time: 20.8 s


,timestamp,power,scene,settings,date
0,2020-01-14 09:53:01,OFF,AUTO,OFF,2020-01-14
1,2020-01-14 09:53:02,OFF,AUTO,OFF,2020-01-14
2,2020-01-14 09:53:03,OFF,AUTO,OFF,2020-01-14
3,2020-01-14 09:53:04,OFF,AUTO,OFF,2020-01-14
4,2020-01-14 09:53:05,OFF,AUTO,OFF,2020-01-14
5,2020-01-14 09:53:06,OFF,AUTO,OFF,2020-01-14
6,2020-01-14 09:53:07,OFF,AUTO,OFF,2020-01-14
7,2020-01-14 09:53:08,OFF,AUTO,OFF,2020-01-14
8,2020-01-14 09:53:09,OFF,AUTO,OFF,2020-01-14
9,2020-01-14 09:53:10,OFF,AUTO,OFF,2020-01-14


In [26]:
ddd.groupby(date)

TypeError: Required argument 'month' (pos 2) not found

In [21]:
dt = ddd.timestamp.iloc[10]
dt

Timestamp('2020-01-14 09:53:11')

In [23]:
dt.date()

datetime.date(2020, 1, 14)

In [64]:
ddd.set_index?

In [17]:
ds['keep_row'] = ds.new_interval.diff(periods=-1)

In [18]:
ds

,timestamp,power,scene,settings,delay,new_interval,keep_row
0,2020-01-15 11:26:01,ON,LIGHT_SHOWER,ON,NaT,1,1.0
1,2020-01-15 11:26:02,ON,LIGHT_SHOWER,ON,00:00:01,0,0.0
2,2020-01-15 11:26:03,ON,LIGHT_SHOWER,ON,00:00:01,0,0.0
3,2020-01-15 11:26:04,ON,LIGHT_SHOWER,ON,00:00:01,0,0.0
4,2020-01-15 11:26:05,ON,LIGHT_SHOWER,ON,00:00:01,0,0.0
5,2020-01-15 11:26:06,ON,LIGHT_SHOWER,ON,00:00:01,0,0.0
6,2020-01-15 11:26:07,ON,LIGHT_SHOWER,ON,00:00:01,0,0.0
7,2020-01-15 11:26:08,ON,LIGHT_SHOWER,ON,00:00:01,0,0.0
8,2020-01-15 11:26:09,ON,LIGHT_SHOWER,ON,00:00:01,0,0.0
9,2020-01-15 11:26:10,ON,LIGHT_SHOWER,ON,00:00:01,0,0.0


In [5]:
data_instructions.target.unique()

array(['POWER_PRESENCE', 'TASK_VERT', 'TASK_HORI', 'SETTINGS',
       'POWER_MANUAL', 'LIGHT_SHOWER', 'RESTART'], dtype=object)

In [14]:
last_ptl_state = data.iloc[-1]
last_ptl_state

power        OFF
scene       AUTO
settings     OFF
Name: 2020-04-10 11:36:27, dtype: object

In [15]:
last_ptl_state.name = until

In [16]:
last_ptl_state

power        OFF
scene       AUTO
settings     OFF
Name: 2020-04-14 00:00:00, dtype: object

In [18]:
data.append(pd.DataFrame([last_ptl_state]))

,power,scene,settings
2020-02-07 09:45:26,OFF,AUTO,OFF
2020-02-07 09:45:27,OFF,AUTO,OFF
2020-02-07 09:45:28,OFF,AUTO,OFF
2020-02-07 09:45:29,OFF,AUTO,OFF
2020-02-07 09:45:30,OFF,AUTO,OFF
2020-02-07 09:45:31,OFF,AUTO,OFF
2020-02-07 09:45:32,OFF,AUTO,OFF
2020-02-07 09:45:33,OFF,AUTO,OFF
2020-02-07 09:45:34,OFF,AUTO,OFF
2020-02-07 09:45:35,OFF,AUTO,OFF


In [5]:
data.loc[data.index.min(), 'power'] = "OFF"

In [8]:
data.loc[data.index.min(), 'mode'] = "AUTO"

In [7]:
data.resample?

In [30]:
%%time
data[(data.index >= day1) & (data.index <= day2) & (data.power == "ON") & (data['mode'] == "AUTO")]

CPU times: user 541 ms, sys: 36 ms, total: 577 ms
Wall time: 583 ms


,power,mode,settings
timestamp,,,
2020-03-12 01:53:26,ON,AUTO,OFF
2020-03-12 01:53:27,ON,AUTO,OFF
2020-03-12 01:53:28,ON,AUTO,OFF
2020-03-12 01:53:29,ON,AUTO,OFF
2020-03-12 01:53:30,ON,AUTO,OFF
2020-03-12 01:53:31,ON,AUTO,OFF
2020-03-12 01:53:32,ON,AUTO,OFF
2020-03-12 01:53:33,ON,AUTO,OFF
2020-03-12 01:53:34,ON,AUTO,OFF


In [22]:
data.index >= day1

array([False, False, False, ...,  True,  True,  True])

In [31]:
dd[(dd.power == "ON") & (dd['mode'] == "LIGHT_SHOWER")]

,target,status,power,mode,settings
timestamp,,,,,
2020-03-12 11:02:35,LIGHT_SHOWER,ON,ON,LIGHT_SHOWER,OFF
2020-03-12 11:02:36,LIGHT_SHOWER,ON,ON,LIGHT_SHOWER,OFF
2020-03-12 11:02:38,LIGHT_SHOWER,ON,ON,LIGHT_SHOWER,OFF
2020-03-12 11:02:40,LIGHT_SHOWER,ON,ON,LIGHT_SHOWER,OFF
2020-03-12 11:02:47,LIGHT_SHOWER,ON,ON,LIGHT_SHOWER,OFF
2020-03-12 11:02:50,LIGHT_SHOWER,ON,ON,LIGHT_SHOWER,OFF
2020-03-12 11:02:52,LIGHT_SHOWER,ON,ON,LIGHT_SHOWER,OFF
2020-03-12 11:02:53,LIGHT_SHOWER,ON,ON,LIGHT_SHOWER,OFF
2020-03-12 11:02:57,LIGHT_SHOWER,ON,ON,LIGHT_SHOWER,OFF


In [19]:
state = None
for i, (index, row) in enumerate(data.iterrows()):
    if i == 0:
        state = State(row.power, row['mode'], row.settings)
        continue
    transition = row
    new_state = state_transitions(state, transition)    
    # update state in current row
    row.power = new_state.power
    row['mode'] = new_state.mode
    row.settings = new_state.settings
    state = new_state

In [20]:
data

,target,status,power,mode,settings
timestamp,,,,,
2020-03-12 00:00:00.000000,POWER_PRESENCE,OFF,OFF,AUTO,OFF
2020-03-12 01:54:52.167262,POWER_PRESENCE,ON,ON,AUTO,OFF
2020-03-12 01:55:22.830804,POWER_PRESENCE,OFF,OFF,AUTO,OFF
2020-03-12 02:03:21.645642,POWER_PRESENCE,ON,ON,AUTO,OFF
2020-03-12 02:03:40.554239,POWER_PRESENCE,OFF,OFF,AUTO,OFF
2020-03-12 08:29:40.462335,POWER_PRESENCE,ON,ON,AUTO,OFF
2020-03-12 08:43:38.797088,POWER_PRESENCE,OFF,OFF,AUTO,OFF
2020-03-12 08:44:39.785051,POWER_PRESENCE,ON,ON,AUTO,OFF
2020-03-12 08:50:03.255949,POWER_PRESENCE,OFF,OFF,AUTO,OFF


In [4]:
def apply_state_transition(window):
    print(window)
    state = window.iloc[0]
    transition = window.iloc[1]
    value = window.iloc[1]
    new_state = state_transitions(state, transition)
    value.power = new_state.power
    value.mode = new_state.mode
    value.settings = new_state.settings
    return value

data.rolling(2).apply(apply_state_transition)

,target,status,power,mode,settings
timestamp,,,,,
2020-03-12 00:00:00.000000,POWER_PRESENCE,OFF,OFF,AUTO,OFF
2020-03-12 01:54:52.167262,POWER_PRESENCE,ON,NaT,NaT,NaT
2020-03-12 01:55:22.830804,POWER_PRESENCE,OFF,NaT,NaT,NaT
2020-03-12 02:03:21.645642,POWER_PRESENCE,ON,NaT,NaT,NaT
2020-03-12 02:03:40.554239,POWER_PRESENCE,OFF,NaT,NaT,NaT
2020-03-12 08:29:40.462335,POWER_PRESENCE,ON,NaT,NaT,NaT
2020-03-12 08:43:38.797088,POWER_PRESENCE,OFF,NaT,NaT,NaT
2020-03-12 08:44:39.785051,POWER_PRESENCE,ON,NaT,NaT,NaT
2020-03-12 08:50:03.255949,POWER_PRESENCE,OFF,NaT,NaT,NaT


In [21]:
def filter_target_column(data: pd.DataFrame, target: str):
    return data[data.target == target].drop(columns='target') \
        .rename(columns=dict(status=target))

In [7]:
# todo add "SETTIGNS"
instruction_targets = ['POWER_MANUAL', 'POWER_PRESENCE', 'LIGHT_SHOWER', 'TASK_VERT', 'TASK_HORI']
split_data = [filter_target_column(data_raw, target) for target in instruction_targets]

In [9]:
from functools import reduce
merged_data = reduce(lambda a, b: pd.merge(a, b, how='outer', on='timestamp'), split_data)

In [10]:
pd.set_option('display.max_rows', 100)
merged_data

,POWER_MANUAL,timestamp,POWER_PRESENCE,LIGHT_SHOWER,TASK_VERT,TASK_HORI
0,NaN,2020-03-12 00:00:00,OFF,NaN,NaN,NaN
1,NaN,2020-03-12 01:54:52,ON,NaN,NaN,NaN
2,NaN,2020-03-12 01:55:23,OFF,NaN,NaN,NaN
3,NaN,2020-03-12 02:03:22,ON,NaN,NaN,NaN
4,NaN,2020-03-12 02:03:41,OFF,NaN,NaN,NaN
5,NaN,2020-03-12 08:29:40,ON,NaN,NaN,NaN
6,NaN,2020-03-12 08:43:39,OFF,NaN,NaN,NaN
7,NaN,2020-03-12 08:44:40,ON,NaN,NaN,NaN
8,NaN,2020-03-12 08:50:03,OFF,NaN,NaN,NaN
9,NaN,2020-03-12 08:57:13,ON,NaN,NaN,NaN


In [23]:
data_ls = data_raw[data_raw.target == 'LIGHT_SHOWER']
data_ls = data_ls.drop(columns=['target']).rename(columns=dict(status="light_shower"))
data_ls

,timestamp,light_shower
40,2020-03-12 11:02:34,ON
41,2020-03-12 11:02:36,OFF
42,2020-03-12 11:02:37,ON
43,2020-03-12 11:02:38,OFF
44,2020-03-12 11:02:40,ON
45,2020-03-12 11:02:41,OFF
52,2020-03-12 11:02:46,ON
53,2020-03-12 11:02:47,OFF
58,2020-03-12 11:02:50,ON
59,2020-03-12 11:02:50,OFF


In [27]:
data_th = data_raw[data_raw.target == 'TASK_HORI']
data_th = data_th.drop(columns=['target']).rename(columns=dict(status="task_horizontal"))
data_th

,timestamp,task_horizontal
47,2020-03-12 11:02:42,ON
49,2020-03-12 11:02:44,ON
51,2020-03-12 11:02:46,ON
55,2020-03-12 11:02:48,ON
57,2020-03-12 11:02:49,ON
63,2020-03-12 11:02:54,ON
65,2020-03-12 11:02:55,ON


In [28]:
data_tv = data_raw[data_raw.target == 'TASK_VERT']
data_tv = data_tv.drop(columns=['target']).rename(columns=dict(status="task_vertical"))
data_tv

,timestamp,task_vertical
46,2020-03-12 11:02:42,ON
48,2020-03-12 11:02:43,ON
50,2020-03-12 11:02:45,ON
54,2020-03-12 11:02:48,ON
56,2020-03-12 11:02:49,ON
62,2020-03-12 11:02:54,ON
64,2020-03-12 11:02:55,ON
68,2020-03-12 11:02:59,ON
69,2020-03-12 11:02:59,OFF


In [16]:
pd.merge?

In [31]:

pd.merge(data_ls, data_th, how='outer', on='timestamp')

,timestamp,light_shower,task_horizontal
0,2020-03-12 11:02:34,ON,NaN
1,2020-03-12 11:02:36,OFF,NaN
2,2020-03-12 11:02:37,ON,NaN
3,2020-03-12 11:02:38,OFF,NaN
4,2020-03-12 11:02:40,ON,NaN
5,2020-03-12 11:02:41,OFF,NaN
6,2020-03-12 11:02:46,ON,ON
7,2020-03-12 11:02:47,OFF,NaN
8,2020-03-12 11:02:50,ON,NaN
9,2020-03-12 11:02:50,OFF,NaN


In [5]:
data

,power_manual,presence,vertical,horizontal,light_shower,settings
timestamp,,,,,,
2020-03-12 00:00:00,0,0,0,0,0,0
2020-03-12 00:00:01,0,0,0,0,0,0
2020-03-12 00:00:02,0,0,0,0,0,0
2020-03-12 00:00:03,0,0,0,0,0,0
2020-03-12 00:00:04,0,0,0,0,0,0
2020-03-12 00:00:05,0,0,0,0,0,0
2020-03-12 00:00:06,0,0,0,0,0,0
2020-03-12 00:00:07,0,0,0,0,0,0
2020-03-12 00:00:08,0,0,0,0,0,0


In [17]:
consecutive = data.shift() == data
data[(consecutive.power & consecutive.vertical & consecutive.horizontal & consecutive.light_shower & consecutive.settings) == False]

,power,vertical,horizontal,light_shower,settings
index,,,,,
2020-03-12 00:00:00,0,0,0,0,0
2020-03-12 01:54:53,1,0,0,0,0
2020-03-12 01:55:23,0,0,0,0,0
2020-03-12 02:03:22,1,0,0,0,0
2020-03-12 02:03:41,0,0,0,0,0
2020-03-12 08:29:41,1,0,0,0,0
2020-03-12 08:43:39,0,0,0,0,0
2020-03-12 08:44:40,1,0,0,0,0
2020-03-12 08:50:04,0,0,0,0,0


In [16]:
dp=data[['power']]

In [24]:
data.applymap(on_off)

,power,vertical,horizontal,light_shower,settings
index,,,,,
2020-03-12 00:00:00,0,0,0,0,0
2020-03-12 00:01:00,0,0,0,0,0
2020-03-12 00:02:00,0,0,0,0,0
2020-03-12 00:03:00,0,0,0,0,0
2020-03-12 00:04:00,0,0,0,0,0
2020-03-12 00:05:00,0,0,0,0,0
2020-03-12 00:06:00,0,0,0,0,0
2020-03-12 00:07:00,0,0,0,0,0
2020-03-12 00:08:00,0,0,0,0,0


In [10]:
import pandas as pd
pd.concat(data, axis=1)

,power,vertical,horizontal,light_shower
index,,,,
2020-03-12 00:00:00,OFF,OFF,OFF,OFF
2020-03-12 00:01:00,OFF,OFF,OFF,OFF
2020-03-12 00:02:00,OFF,OFF,OFF,OFF
2020-03-12 00:03:00,OFF,OFF,OFF,OFF
2020-03-12 00:04:00,OFF,OFF,OFF,OFF
2020-03-12 00:05:00,OFF,OFF,OFF,OFF
2020-03-12 00:06:00,OFF,OFF,OFF,OFF
2020-03-12 00:07:00,OFF,OFF,OFF,OFF
2020-03-12 00:08:00,OFF,OFF,OFF,OFF


In [18]:
data_vert.reset_index().drop_duplicates(subset=['index']).set_index('index')

,power
index,
2020-03-12 00:00:00.000000,OFF
2020-03-12 01:54:52.167262,ON
2020-03-12 01:55:22.830804,OFF
2020-03-12 02:03:21.645642,ON
2020-03-12 02:03:40.554239,OFF
2020-03-12 08:29:40.462335,ON
2020-03-12 08:43:38.797088,OFF
2020-03-12 08:44:39.785051,ON
2020-03-12 08:50:03.255949,OFF


In [5]:
data_vert.resample('1s').ffill()

,vertical
2020-03-12 00:00:00,OFF
2020-03-12 00:00:01,OFF
2020-03-12 00:00:02,OFF
2020-03-12 00:00:03,OFF
2020-03-12 00:00:04,OFF
2020-03-12 00:00:05,OFF
2020-03-12 00:00:06,OFF
2020-03-12 00:00:07,OFF
2020-03-12 00:00:08,OFF
2020-03-12 00:00:09,OFF


In [27]:
def expand_dataframe(data, until: datetime):
    # get last data point
    last_value = dp.iloc[-1].copy()
    # change timestamp
    last_value.name = until
    return pd.concat([data, pd.DataFrame([last_value])], axis=0)

In [37]:
data.target.unique()

array(['POWER', 'LIGHT_SHOWER', 'TASK_VERT', 'TASK_HORI'], dtype=object)

In [4]:
dp = data[data.target == "TASK_HORI"]
last_value = dp.iloc[-1].copy()
last_value.name = until

In [7]:
expand_mode_dataframe(data[data.target == "TASK_HORI"], since, until)

,target,status
2020-03-12 00:00:00.000000,TASK_HORI,OFF
2020-03-12 11:02:42.393790,TASK_HORI,ON
2020-03-12 11:02:44.405254,TASK_HORI,ON
2020-03-12 11:02:46.063290,TASK_HORI,ON
2020-03-12 11:02:48.218812,TASK_HORI,ON
2020-03-12 11:02:49.294226,TASK_HORI,ON
2020-03-12 11:02:54.343672,TASK_HORI,ON
2020-03-12 11:02:55.420499,TASK_HORI,ON
2020-03-13 00:00:00.000000,TASK_HORI,ON


In [34]:
expand_dataframe(dp, until).resample("1s").ffill()

,target,status
2020-03-12 11:02:41,NaN,NaN
2020-03-12 11:02:42,TASK_VERT,ON
2020-03-12 11:02:43,TASK_VERT,ON
2020-03-12 11:02:44,TASK_VERT,ON
2020-03-12 11:02:45,TASK_VERT,ON
2020-03-12 11:02:46,TASK_VERT,ON
2020-03-12 11:02:47,TASK_VERT,ON
2020-03-12 11:02:48,TASK_VERT,ON
2020-03-12 11:02:49,TASK_VERT,ON
2020-03-12 11:02:50,TASK_VERT,ON


In [21]:
import pandas as pd
pd.DataFrame([last_value])

,target,status
2020-03-13,POWER,OFF


In [11]:
test = dp.iloc[-1]

In [12]:
type(test)

pandas.core.series.Series

In [14]:
test.timestamp = until

In [16]:
test.name = until

In [17]:
test

target    POWER
status      OFF
Name: 2020-03-13 00:00:00, dtype: object

In [21]:
data.resample('1s').fillna(method='ffill')

,target,status
timestamp,,
2020-03-12 01:54:52,NaN,NaN
2020-03-12 01:54:53,POWER,ON
2020-03-12 01:54:54,POWER,ON
2020-03-12 01:54:55,POWER,ON
2020-03-12 01:54:56,POWER,ON
2020-03-12 01:54:57,POWER,ON
2020-03-12 01:54:58,POWER,ON
2020-03-12 01:54:59,POWER,ON
2020-03-12 01:55:00,POWER,ON


In [5]:
%%time
data = get_sensor_data(device, (since, until), rule="1s")

CPU times: user 267 ms, sys: 218 ms, total: 484 ms
Wall time: 2min 6s


In [6]:
data

,temperature,pressure,humidity,gas,brightness_lv,brightness_lh,brightness_rv,brightness_rh
timestamp,,,,,,,,
2020-03-12 00:00:00,23.97,918.43,22.34,509.00,0,0,0,0
2020-03-12 00:00:01,23.97,918.43,22.34,509.00,0,0,0,0
2020-03-12 00:00:02,23.97,918.43,22.34,509.00,0,0,0,0
2020-03-12 00:00:03,23.97,918.43,22.34,509.00,0,0,0,0
2020-03-12 00:00:04,23.97,918.42,22.36,509.00,0,0,0,0
2020-03-12 00:00:05,23.97,918.42,22.36,509.00,0,0,0,0
2020-03-12 00:00:06,23.97,918.42,22.36,509.00,0,0,0,0
2020-03-12 00:00:07,23.97,918.42,22.36,509.00,0,0,0,0
2020-03-12 00:00:08,23.97,918.42,22.36,509.00,0,0,0,0


In [4]:
interval = [since, until]
isinstance(since, datetime)

True

In [4]:
ti = TimeInterval(since, until)

In [6]:
ti.length

datetime.timedelta(0, 10800)

In [ ]:
# 1 worker:  Wall time: 1min 28s
# 8 workers: Wall time: 51.1 s

In [8]:
%%time


CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 7.15 µs


'TEMPERATURE'

In [4]:
data

,temperature,pressure,humidity,gas,brightness_lv,brightness_lh,brightness_rv,brightness_rh
timestamp,,,,,,,,
2020-03-12 09:00:00,26.494333,916.803833,22.259667,404.647667,997.216667,429.333333,422.933333,271.116667
2020-03-12 09:01:00,26.517000,916.781167,22.223000,406.392833,933.250000,398.850000,410.183333,255.883333
2020-03-12 09:02:00,26.540667,916.740500,22.173000,407.092500,945.716667,396.633333,426.750000,252.566667
2020-03-12 09:03:00,26.564333,916.666667,22.137500,407.292667,836.700000,351.066667,398.133333,227.566667
2020-03-12 09:04:00,26.588000,916.600333,22.082333,409.256667,924.900000,396.100000,421.583333,250.016667
2020-03-12 09:05:00,26.611833,916.568167,22.039000,409.419000,914.250000,401.250000,417.000000,253.300000
2020-03-12 09:06:00,26.635000,916.605333,22.021333,411.523667,861.366667,391.966667,390.433333,248.100000
2020-03-12 09:07:00,26.651333,916.654000,21.980000,409.944667,828.533333,372.883333,393.850000,238.516667
2020-03-12 09:08:00,26.672500,916.680000,21.941000,412.747833,922.750000,411.266667,438.733333,258.100000


In [15]:
data.resample('1H').mean()

,temperature,pressure,humidity,gas,brightness_lv,brightness_lh,brightness_rv,brightness_rh
timestamp,,,,,,,,
2020-03-12 09:00:00,27.115486,916.565106,21.496764,423.879253,831.664167,438.644722,454.109167,286.722222
2020-03-12 10:00:00,28.226750,916.060311,20.275850,444.785586,793.533333,471.580556,548.271111,344.072500
2020-03-12 11:00:00,29.293792,915.565003,18.954322,451.968917,785.121944,488.845556,591.907778,353.944722
2020-03-12 12:00:00,29.840000,915.080000,18.390000,449.220000,793.000000,528.000000,590.000000,366.000000


In [3]:
data = Connectivity.connection_times(since, until, device)
connection_data = data[data.connected==1].reset_index()[["begin", "end"]]
connection_data

,begin,end
0,2020-03-12 11:01:27.834999,2020-03-12 12:15:41.037585
1,2020-03-12 12:18:17.408448,2020-03-12 12:20:18.027845
2,2020-03-12 12:23:48.719316,2020-03-12 15:28:20.845609
3,2020-03-12 15:31:09.530427,2020-03-13 00:00:00.000000


In [4]:
data = PresenceDetectorStatistics().on_off_timeseries(since, until, device)
presence_data = data[data.value == 1].reset_index()[['begin', 'end']]
presence_data

,begin,end
0,2020-03-12 01:54:52.167262,2020-03-12 01:55:22.830804
1,2020-03-12 02:03:21.645642,2020-03-12 02:03:40.554239
2,2020-03-12 08:29:40.462335,2020-03-12 08:43:38.797088
3,2020-03-12 08:44:39.785051,2020-03-12 08:50:03.255949
4,2020-03-12 08:57:13.152302,2020-03-12 09:12:44.638700
5,2020-03-12 09:13:30.258904,2020-03-12 09:33:01.231312
6,2020-03-12 09:37:22.225314,2020-03-12 09:38:27.977446
7,2020-03-12 09:38:45.604175,2020-03-12 09:39:22.195649
8,2020-03-12 09:40:49.781471,2020-03-12 09:52:13.633493
9,2020-03-12 09:53:07.025995,2020-03-12 09:59:16.342491


In [5]:
intervals = intersect_intervals(presence_data, connection_data)
intervals = remove_short_intervals(intervals, length=timedelta(minutes=1))
intervals

,begin,end
1,2020-03-12 11:02:27.776184,2020-03-12 12:15:41.037585
2,2020-03-12 12:18:17.408448,2020-03-12 12:20:18.027845
3,2020-03-12 12:23:48.719316,2020-03-12 12:29:24.135825
4,2020-03-12 12:54:39.705964,2020-03-12 13:19:57.913696
5,2020-03-12 13:20:30.002676,2020-03-12 13:49:34.438984
6,2020-03-12 13:49:46.009046,2020-03-12 13:54:25.240610
7,2020-03-12 14:12:53.877655,2020-03-12 15:10:23.599668
8,2020-03-12 15:12:15.618124,2020-03-12 15:28:20.845609
9,2020-03-12 15:31:09.530427,2020-03-12 15:48:40.993915
10,2020-03-12 15:48:56.230357,2020-03-12 16:00:10.028289


In [ ]:
%%time
data = get_sensor_data(device, intervals)

Get data for interval: 2020-03-12 11:02:27.776184 - 2020-03-12 12:15:41.037585
Get data for interval: 2020-03-12 12:18:17.408448 - 2020-03-12 12:20:18.027845
Get data for interval: 2020-03-12 12:23:48.719316 - 2020-03-12 12:29:24.135825
Get data for interval: 2020-03-12 12:54:39.705964 - 2020-03-12 13:19:57.913696
Get data for interval: 2020-03-12 13:20:30.002676 - 2020-03-12 13:49:34.438984
Get data for interval: 2020-03-12 13:49:46.009046 - 2020-03-12 13:54:25.240610
Get data for interval: 2020-03-12 14:12:53.877655 - 2020-03-12 15:10:23.599668
Get data for interval: 2020-03-12 15:12:15.618124 - 2020-03-12 15:28:20.845609
